# Clutter Channel / Clutter Map Estimation
The clutter map of a radar system is a representation of the stationary objects that create unwanted echoes, in this case the reflections from ceiling, walls and stationary objects in the lab room. We estimate the clutter map directly in the frequency-antenna domain, so we are effectively estimating a "clutter channel" $\bar H$, *separately for each transmitter*. This channel can be interpreted as the channel measurement we would expect for an empty room without any target.

By subtracting the clutter map from measured channels, we can separate the disturbance / reflection generated by the mobile targets from the clutter, which makes radar signal processing possible in the first place.

Here, we use the algorithm from the paper: M. Henninger, S. Mandelli, A. Grudnitsky, T. Wild, S. ten Brink: "CRAP: Clutter Removal with Acquisitions Under Phase Noise"
This algorithm estimates the principal components of the clutter subspace and projects each datapoint onto the clutter subspace to determine the clutter which is subsequently removed by subtraction. Here, we assume a constant clutter order $L = 2$, which is an arbitrary choice that seems to work well enough.

In [1]:
from tqdm.auto import tqdm
import espargos_0007
import numpy as np
import CRAP
import os

2025-04-10 13:11:13.849305: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-10 13:11:13.869731: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-10 13:11:13.876000: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-10 13:11:13.891522: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-10 13:11:14.848906: W tensorflow/compiler/tf2

In [2]:
# Loading all the datasets can take some time...
training_set_robot = espargos_0007.load_dataset(espargos_0007.TRAINING_SET_ROBOT_FILES)
test_set_robot = espargos_0007.load_dataset(espargos_0007.TEST_SET_ROBOT_FILES)
test_set_human = espargos_0007.load_dataset(espargos_0007.TEST_SET_HUMAN_FILES)

all_datasets = training_set_robot + test_set_robot + test_set_human

  0%|          | 0/3 [00:00<?, ?it/s]

Loading espargos-0007/espargos-0007-meanders-nw-se-1.tfrecords


I0000 00:00:1744290676.162237   18944 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1744290676.211899   18944 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1744290676.212290   18944 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1744290676.215883   18944 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Loading espargos-0007/espargos-0007-meanders-sw-ne-1.tfrecords


2025-04-10 13:11:38.902610: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

  0%|          | 0/1 [00:00<?, ?it/s]

Loading espargos-0007/espargos-0007-randomwalk-2.tfrecords


2025-04-10 13:11:53.851626: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


  0%|          | 0/1 [00:00<?, ?it/s]

Loading espargos-0007/espargos-0007-human-helmet-randomwalk-1.tfrecords


### Clutter Channel Estimation using CRAP algorithm

In [3]:
clutter_acquisitions_by_dataset = dict()

for dataset in tqdm(all_datasets):
    print(f"Computing clutter channel for dataset {dataset["filename"]}")
    clutter_acquisitions = []
    for tx_idx, mac in enumerate(tqdm(dataset["unique_macs"])):
        # Filter CSI by transmitter index (= transmitter MAC address)
        csi_filtered = []
        for i in range(dataset["csi_freq_domain"].shape[0]):
            if dataset["source_macs"][i] == mac:
                csi_filtered.append(dataset["csi_freq_domain"][i])
    
        csi_filtered = np.asarray(csi_filtered)
        clutter_acquisitions.append(CRAP.acquire_clutter(csi_filtered, order = 2))

    clutter_acquisitions_by_dataset[dataset["filename"]] = np.asarray(clutter_acquisitions)

  0%|          | 0/5 [00:00<?, ?it/s]

Computing clutter channel for dataset espargos-0007/espargos-0007-meanders-nw-se-1.tfrecords


  0%|          | 0/4 [00:00<?, ?it/s]

Computing clutter channel for dataset espargos-0007/espargos-0007-meanders-sw-ne-1.tfrecords


  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

Computing clutter channel for dataset espargos-0007/espargos-0007-randomwalk-2.tfrecords


  0%|          | 0/4 [00:00<?, ?it/s]

Computing clutter channel for dataset espargos-0007/espargos-0007-human-helmet-randomwalk-1.tfrecords


  0%|          | 0/4 [00:00<?, ?it/s]

### Store clutter channels to file

In [4]:
!mkdir -p clutter_channel_estimates
for filename, clutter_acquisitions_by_tx in clutter_acquisitions_by_dataset.items():
    np.save(os.path.join("clutter_channel_estimates", os.path.basename(filename)), np.asarray(clutter_acquisitions_by_tx))

### Optional: Estimate Clutter Power vs Target Reflection Power

Note that estimate for non-clutter power contains both the power of the reflected / scattered / blocked components, but also noise power!

In [5]:
for dataset in tqdm(all_datasets):
    print(f"Estimating clutter channel power for dataset {dataset["filename"]}")
    for tx_idx, mac in enumerate(tqdm(dataset["unique_macs"])):
        # Filter CSI by transmitter index (= transmitter MAC address)
        csi_filtered = []
        for i in range(dataset["csi_freq_domain"].shape[0]):
            if dataset["source_macs"][i] == mac:
                csi_filtered.append(dataset["csi_freq_domain"][i])
    
        csi_cluttered = np.asarray(csi_filtered)
        csi_noclutter = CRAP.remove_clutter(csi_cluttered, clutter_acquisitions_by_dataset[dataset["filename"]][tx_idx])

        csi_cluttered_power = np.mean(np.sum(np.abs(csi_cluttered)**2, axis = (1, 2, 3, 4)), axis = 0)
        csi_noclutter_power = np.mean(np.sum(np.abs(csi_noclutter)**2, axis = (1, 2, 3, 4)), axis = 0)
        clutter_only_power = csi_cluttered_power - csi_noclutter_power

        print(f"======== Transmitter {tx_idx}: {mac} ========")
        print(f"                Datapoint mean power: {10 * np.log10(csi_cluttered_power):.2f} dB")
        print(f"Datapoint mean power without clutter: {10 * np.log10(csi_noclutter_power):.2f} dB")
        print(f"                       Clutter power: {10 * np.log10(clutter_only_power):.2f} dB")
        print(f" Share of non-clutter power of total: {10 * np.log10(csi_noclutter_power / csi_cluttered_power):.2f} dB")

  0%|          | 0/5 [00:00<?, ?it/s]

Estimating clutter channel power for dataset espargos-0007/espargos-0007-meanders-nw-se-1.tfrecords


  0%|          | 0/4 [00:00<?, ?it/s]

======== Transmitter 0: 0a:ee:f5:00:00:01 ========
                Datapoint mean power: 30.93 dB
Datapoint mean power without clutter: 14.07 dB
                       Clutter power: 30.84 dB
 Share of non-clutter power of total: -16.86 dB
======== Transmitter 1: 0a:ee:f5:00:00:02 ========
                Datapoint mean power: 25.47 dB
Datapoint mean power without clutter: 12.11 dB
                       Clutter power: 25.26 dB
 Share of non-clutter power of total: -13.36 dB
======== Transmitter 2: 0a:ee:f5:00:00:03 ========
                Datapoint mean power: 25.88 dB
Datapoint mean power without clutter: 12.15 dB
                       Clutter power: 25.69 dB
 Share of non-clutter power of total: -13.72 dB
======== Transmitter 3: 0a:ee:f5:00:00:04 ========
                Datapoint mean power: 27.49 dB
Datapoint mean power without clutter: 12.84 dB
                       Clutter power: 27.33 dB
 Share of non-clutter power of total: -14.65 dB
Estimating clutter channel power for dat

  0%|          | 0/4 [00:00<?, ?it/s]

...

======== Transmitter 0: 0a:ee:f5:00:00:01 ========
                Datapoint mean power: 30.73 dB
Datapoint mean power without clutter: 13.77 dB
                       Clutter power: 30.64 dB
 Share of non-clutter power of total: -16.95 dB
======== Transmitter 1: 0a:ee:f5:00:00:02 ========
                Datapoint mean power: 25.40 dB
Datapoint mean power without clutter: 12.01 dB
                       Clutter power: 25.19 dB
 Share of non-clutter power of total: -13.38 dB
======== Transmitter 2: 0a:ee:f5:00:00:03 ========
                Datapoint mean power: 25.87 dB
Datapoint mean power without clutter: 11.78 dB
                       Clutter power: 25.70 dB
 Share of non-clutter power of total: -14.09 dB
======== Transmitter 3: 0a:ee:f5:00:00:04 ========
                Datapoint mean power: 27.52 dB
Datapoint mean power without clutter: 12.90 dB
                       Clutter power: 27.37 dB
 Share of non-clutter power of total: -14.63 dB
Estimating clutter channel power for dat

  0%|          | 0/4 [00:00<?, ?it/s]

======== Transmitter 0: 0a:ee:f5:00:00:01 ========
                Datapoint mean power: 30.75 dB
Datapoint mean power without clutter: 13.58 dB
                       Clutter power: 30.66 dB
 Share of non-clutter power of total: -17.16 dB
======== Transmitter 1: 0a:ee:f5:00:00:02 ========
                Datapoint mean power: 25.52 dB
Datapoint mean power without clutter: 11.81 dB
                       Clutter power: 25.33 dB
 Share of non-clutter power of total: -13.71 dB
======== Transmitter 2: 0a:ee:f5:00:00:03 ========
                Datapoint mean power: 25.81 dB
Datapoint mean power without clutter: 11.82 dB
                       Clutter power: 25.64 dB
 Share of non-clutter power of total: -13.99 dB
======== Transmitter 3: 0a:ee:f5:00:00:04 ========
                Datapoint mean power: 27.26 dB
Datapoint mean power without clutter: 12.16 dB
                       Clutter power: 27.12 dB
 Share of non-clutter power of total: -15.10 dB
Estimating clutter channel power for dat

  0%|          | 0/4 [00:00<?, ?it/s]

======== Transmitter 0: 0a:ee:f5:00:00:01 ========
                Datapoint mean power: 30.99 dB
Datapoint mean power without clutter: 14.79 dB
                       Clutter power: 30.88 dB
 Share of non-clutter power of total: -16.20 dB
======== Transmitter 1: 0a:ee:f5:00:00:02 ========
                Datapoint mean power: 25.48 dB
Datapoint mean power without clutter: 11.38 dB
                       Clutter power: 25.31 dB
 Share of non-clutter power of total: -14.11 dB
======== Transmitter 2: 0a:ee:f5:00:00:03 ========
                Datapoint mean power: 26.15 dB
Datapoint mean power without clutter: 11.86 dB
                       Clutter power: 25.99 dB
 Share of non-clutter power of total: -14.29 dB
======== Transmitter 3: 0a:ee:f5:00:00:04 ========
                Datapoint mean power: 27.12 dB
Datapoint mean power without clutter: 12.02 dB
                       Clutter power: 26.98 dB
 Share of non-clutter power of total: -15.10 dB


### Optional: Plot Principal Component of Clutter Channels

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def channel_plot(csi_datapoint, suptitle = None, title = None):
    plt.figure(figsize = (8, 6))
    plt.subplot(211)
    plt.suptitle("Channel State Information" if suptitle is None else suptitle)
    if title is not None:
        plt.title(title)
    plt.xlabel("Subcarrier $i$")
    plt.ylabel("Referenced channel coeff.,\n abs. value $|h_i|_{dB}$ [dB]")
    for b in range(csi_datapoint.shape[0]):
        for r in range(csi_datapoint.shape[1]):
            for c in range(csi_datapoint.shape[2]):
                plt.plot(20 * np.log10(np.abs(csi_datapoint[b,r,c,:])))

    plt.subplot(212)
    plt.xlabel("Subcarrier $i$")
    plt.ylabel("Referenced channel coeff.,\nphase shift $arg(h_i)$")
    for b in range(csi_datapoint.shape[0]):
        for r in range(csi_datapoint.shape[1]):
            for c in range(csi_datapoint.shape[2]):
                plt.plot(np.unwrap(np.angle(csi_datapoint[b,r,c,:])))

    plt.tight_layout()
    plt.show()

In [ ]:
for filename, clutter_acquisitions_by_tx in clutter_acquisitions_by_dataset.items():
    for tx_idx in range(clutter_acquisitions_by_tx.shape[0]):
        clutter_basis = np.reshape(clutter_acquisitions_by_tx[tx_idx], (espargos_0007.ARRAY_COUNT, espargos_0007.ROW_COUNT, espargos_0007.COL_COUNT, espargos_0007.SUBCARRIER_COUNT, -1))
        channel_plot(clutter_basis[:,:,:,:,0], suptitle = "Principal Component of Clutter Channel Estimate", title = f"Transmitter {tx_idx} in {os.path.basename(filename)}")